In [1]:
import requests
import pandas as pd
import datetime as dt
import numpy as np

In [2]:
matches = pd.read_csv('matches.csv')
pd.set_option('display.max_columns', None)

In [3]:
#Sends request to get past matches

def pastMatchesRequest(date_from, date_to, competition_id, page):
    url = "https://live-score-api.p.rapidapi.com/scores/history.json"

    headers = {
        'x-rapidapi-key': "ce0be8d538msh30ecb1b4c0fca98p105373jsn1e40f1d8d1ec",
        'x-rapidapi-host': "live-score-api.p.rapidapi.com",
        }

    params = {
        'key': "MC8uSjXafFUAGJqM",
        'secret': "xRuVIJwAPNVMqsYc46HXqF7Fk2f1MGNj",
        'from': date_from,
        'to': date_to,
        'competition_id': competition_id,
        'page': page
    }

    return requests.request("GET", url, params = params,headers=headers).json()

In [4]:
#Sends request to get stats from a specific match

def matchStatsRequest(match_id):
    url = "https://live-score-api.p.rapidapi.com/matches/stats.json"

    headers = {
        'x-rapidapi-key': "ce0be8d538msh30ecb1b4c0fca98p105373jsn1e40f1d8d1ec",
        'x-rapidapi-host': "live-score-api.p.rapidapi.com",
        }

    params = {
        'key': "MC8uSjXafFUAGJqM",
        'secret': "xRuVIJwAPNVMqsYc46HXqF7Fk2f1MGNj",
        'match_id': match_id
    }

    return requests.request("GET", url, params = params,headers=headers).json()

In [5]:
#Stores information from 'Past matches' on a list

def addMatchInfo(response, continent, country, league_name, type, tier):
    for info in response['data']['match']:
        if(info['ht_score'] == ''):
            info['ht_score'] = '0 - 0'
            
        if(info['scheduled'] is None or info['scheduled'] == ""):
            matches_list.append([
                info['id'],
                info['home_id'],
                info['home_name'],
                info['away_id'],
                info['away_name'],
                info['score'].split(' -')[0],
                info['score'].split('- ')[1],
                info['ht_score'].split(' -')[0],
                info['ht_score'].split('- ')[1],
                info['location'],
                'City',
                country,
                continent,
                league_name,
                type,
                tier,
                np.nan,
                np.nan,
                info['date'].split('-')[2],
                dt.date(int(info['date'].split('-')[0]),int(info['date'].split('-')[1]),int(info['date'].split('-')[2])).strftime("%A"),
                info['date'].split('-')[1],
                info['date'].split('-')[0],
                'F'
            ])
        else:
            matches_list.append([
                info['id'],
                info['home_id'],
                info['home_name'],
                info['away_id'],
                info['away_name'],
                info['score'].split(' -')[0],
                info['score'].split('- ')[1],
                info['ht_score'].split(' -')[0],
                info['ht_score'].split('- ')[1],
                info['location'],
                'City',
                country,
                continent,
                league_name,
                type,
                tier,
                info['scheduled'].split(':')[0],
                info['scheduled'].split(':')[1],
                info['date'].split('-')[2],
                dt.date(int(info['date'].split('-')[0]),int(info['date'].split('-')[1]),int(info['date'].split('-')[2])).strftime("%A"),
                info['date'].split('-')[1],
                info['date'].split('-')[0],
                'F'
            ])

In [6]:
#Updates df with 'Match stats' information

def addMatchStats(rowIndex, matchId):
    response = matchStatsRequest(matchId)
    if(response['success'] == False):
        return False
    if(response['data'] != []):
        matches.at[rowIndex,'home_yc'] = response['data']['yellow_cards'].split(':')[0]
        matches.at[rowIndex,'away_yc'] = response['data']['yellow_cards'].split(':')[1]
        matches.at[rowIndex,'home_rc'] = response['data']['red_cards'].split(':')[0]
        matches.at[rowIndex,'away_rc'] = response['data']['red_cards'].split(':')[1]
        if(response['data']['possesion'] is None):
            matches.at[rowIndex,'home_possession'] = np.nan
            matches.at[rowIndex,'away_possession'] = np.nan
        else:
            matches.at[rowIndex,'home_possession'] = response['data']['possesion'].split(':')[0]
            matches.at[rowIndex,'away_possession'] = response['data']['possesion'].split(':')[1]
            
        if(response['data']['attempts_on_goal'] is None):
            matches.at[rowIndex,'home_shots'] = np.nan
            matches.at[rowIndex,'away_shots'] = np.nan 
        else:
            matches.at[rowIndex,'home_shots'] = response['data']['attempts_on_goal'].split(':')[0] 
            matches.at[rowIndex,'away_shots'] = response['data']['attempts_on_goal'].split(':')[1] 
            
        if(response['data']['shots_on_target'] is not None):
            matches.at[rowIndex,'home_shots_on_target'] = response['data']['shots_on_target'].split(':')[0]
            matches.at[rowIndex,'away_shots_on_target'] = response['data']['shots_on_target'].split(':')[1]
        elif(response['data']['attempts_on_goal'] is not None and response['data']['shots_off_target'] is not None):
            matches.at[rowIndex,'home_shots_on_target'] = int(response['data']['attempts_on_goal'].split(':')[0]) - int(response['data']['shots_off_target'].split(':')[0])
            matches.at[rowIndex,'away_shots_on_target'] = int(response['data']['attempts_on_goal'].split(':')[1]) - int(response['data']['shots_off_target'].split(':')[1])
        else:
            matches.at[rowIndex,'home_shots_on_target'] = np.nan
            matches.at[rowIndex,'away_shots_on_target'] = np.nan 
       
        if(response['data']['fauls'] is None):
            matches.at[rowIndex,'home_fouls'] = np.nan
            matches.at[rowIndex,'away_fouls'] = np.nan
        else:
            matches.at[rowIndex,'home_fouls'] = response['data']['fauls'].split(':')[0]
            matches.at[rowIndex,'away_fouls'] = response['data']['fauls'].split(':')[1]
        
        if(response['data']['attacks'] is None):
            matches.at[rowIndex,'home_attacks'] = np.nan
            matches.at[rowIndex,'away_attacks'] = np.nan
        else:
            matches.at[rowIndex,'home_attacks'] = response['data']['attacks'].split(':')[0]
            matches.at[rowIndex,'away_attacks'] = response['data']['attacks'].split(':')[1]
    matches.at[rowIndex,'got_stats'] = 'T'
    return True

In [7]:
#Gets past matches and stores it on a df

def getMatches(date_from, date_to, competition_id, continent, country, league_name, type, tier):
    page = 1
    
    response = pastMatchesRequest(date_from, date_to, competition_id, page)
    addMatchInfo(response, continent, country, league_name, type, tier)
    
    total_pages = response['data']['total_pages']
    page += 1
    
    while(page <= total_pages):
        response = pastMatchesRequest(date_from, date_to, competition_id, page)
        addMatchInfo(response, continent, country, league_name, type, tier)
        page += 1
    
    return pd.DataFrame(matches_list, columns =['match_id','home_id','home_name','away_id','away_name','home_ft_goals','away_ft_goals','home_ht_goals','away_ht_goals','stadium_name','city','country','continent','competition_name','type','tier','hour','minute','day','dayofweek','month','year','got_stats'])

In [8]:
#Gets match stats from top to bottom until an error is found

def getMatchStats():
    success = True
    
    while(success):
        rowIndex = matches[matches['got_stats'] == 'F'].head(1).tail(1).index
        matchId = matches[matches['got_stats'] == 'F'].head(1).tail(1)['match_id']
        success = addMatchStats(rowIndex, matchId)

In [9]:
def updateLocation(team_id, team_name, stadium_name, city):
    matches.loc[matches.home_id == team_id, 'home_name'] = team_name   
    matches.loc[matches.away_id == team_id, 'away_name'] = team_name 
    matches.loc[matches.home_id == team_id, 'stadium_name'] = stadium_name
    matches.loc[matches.home_id == team_id, 'city'] = city

In [10]:
def updateTeamName(old_name, new_name):
    matches.loc[matches.home_name == old_name, 'home_name'] = new_name
    matches.loc[matches.away_name == old_name, 'away_name'] = new_name

# Competition Ids

1 - Bundesliga (Both)

93 - 2.Bundesliga (Both)

167 - DFB Pokal (Last)

2 - Premier League (Both)

77 - Championship (Both)

150 - EFL Cup (Last)

152 - FA Cup (Last)

3 - La Liga (Both)

79 - Spanish Segunda Division (Both)

334 - Copa Del Rey (Last)

4 - Serie A (Both)

87 - Serie B (Both)

179 - Coppa Italia (Last)

5 - Ligue 1 (Both)

162 - Coupe de France (Last)

163 - Coupe Ligue (Last)

6 - Turkish Super Lig (Both)

347 - Fortis Turkish Cup (Last) 

7 - Russian Premier League (Both)

311 - Kubok (Last) 

8 - Primeira Liga (Both)

92 - Segunda Liga (Both)

212 - Taça de Portugal (Last)

196 - Eredivisie

9 - Greek Super League (Both)

64 - Ukranian Premier League (Both)

244 - Champions League (Both)
 
245 - Europa League (Both)

24 - Brasileirão (Both)

23 - Liga Professional (Both)

329 - Copa Libertadores (Both)

76 - MLS (Both)

45 - Liga MX (Both)

26 - Chinese Super League (Both)

28 - J-League (Both)

67 - A-League (Last)

42 - Tunisian Ligue 1 (Both)

In [19]:
matches_list = []
df = getMatches('2019-07-01','2020-12-21', 67, 'Oceania', 'Australia', 'A-League', 'League', 1)
df

,match_id,home_id,home_name,away_id,away_name,home_ft_goals,away_ft_goals,home_ht_goals,away_ht_goals,stadium_name,city,country,continent,competition_name,type,tier,hour,minute,day,dayofweek,month,year,got_stats
0,165371,1006,Adelaide United,481,Sydney FC,2,3,1,2,None,City,Australia,Oceania,A-League,League,1,08,30,11,Friday,10,2019,F
1,165462,199,Western Sydney Wanderers FC,3135,Central Coast Mariners,2,1,1,1,Bankwest Stadium,City,Australia,Oceania,A-League,League,1,06,00,12,Saturday,10,2019,F
2,165468,1054,Melbourne Victory,200,Melbourne City FC,0,0,0,0,AAMI Park,City,Australia,Oceania,A-League,League,1,08,30,12,Saturday,10,2019,F
3,165749,202,Wellington Phoenix,4545,Western United,0,1,0,1,Westpac Stadium,City,Australia,Oceania,A-League,League,1,05,00,13,Sunday,10,2019,F
4,165760,201,Perth Glory,3185,Brisbane Roar FC,1,1,1,0,None,City,Australia,Oceania,A-League,League,1,07,00,13,Sunday,10,2019,F
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141,204837,202,Wellington Phoenix,201,Perth Glory,0,1,0,1,Westpac Stadium,City,Australia,Oceania,A-League,League,1,07,00,22,Saturday,08,2020,F
142,205208,3185,Brisbane Roar FC,4545,Western United,0,1,0,1,Suncorp Stadium,City,Australia,Oceania,A-League,League,1,08,00,23,Sunday,08,2020,F
143,205619,200,Melbourne City FC,4545,Western United,2,0,0,0,AAMI Park,City,Australia,Oceania,A-League,League,1,07,00,26,Wednesday,08,2020,F
144,205629,481,Sydney FC,201,Perth Glory,2,0,2,0,Netstrata Jubilee Stadium,City,Australia,Oceania,A-League,League,1,10,10,26,Wednesday,08,2020,F


In [20]:
matches = matches.append(df, ignore_index = True)
matches.to_csv('matches.csv', index=False);

In [16]:
getMatchStats()
matches[matches['got_stats'] == 'F']
matches.to_csv('matches.csv', index=False);

In [580]:
#updateTeamName('SC Paderborn 07', 'Paderborn')
matches.at[8806,'away_name'] = 'CD Nacional'
#matches.at[9666,'away_name'] = 'Farense'
#matches.loc[matches.competition_name == 'Brasileirão', 'country'] = 'Brazil'

In [141]:
#matches.loc[matches.away_name == 'Puebla Club De Futbol', 'away_name'] = 'Puebla'
matches[matches['home_id'] == 3185] 

,match_id,home_id,home_name,away_id,away_name,home_ft_goals,away_ft_goals,home_ht_goals,away_ht_goals,stadium_name,city,country,continent,competition_name,type,tier,hour,minute,day,dayofweek,month,year,got_stats,home_yc,away_yc,home_rc,away_rc,home_possession,away_possession,home_shots,away_shots,home_shots_on_target,away_shots_on_target,home_fouls,away_fouls,home_attacks,away_attacks
9899,167838.0,3185.0,Brisbane Roar FC,1054.0,Melbourne Victory,0,1,0,1,Suncorp Stadium,City,Australia,Oceania,A-League,League,1.0,08,30,25,Friday,10,2019,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9920,172939.0,3185.0,Brisbane Roar FC,200.0,Melbourne City FC,4,3,0,2,Suncorp Stadium,City,Australia,Oceania,A-League,League,1.0,05,00,17,Sunday,11,2019,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9928,174655.0,3185.0,Brisbane Roar FC,3135.0,Central Coast Mariners,2,0,0,0,Suncorp Stadium,City,Australia,Oceania,A-League,League,1.0,06,00,30,Saturday,11,2019,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9937,176706.0,3185.0,Brisbane Roar FC,4545.0,Western United,0,2,0,2,Suncorp Stadium,City,Australia,Oceania,A-League,League,1.0,08,30,13,Friday,12,2019,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9959,179227.0,3185.0,Brisbane Roar FC,200.0,Melbourne City FC,2,2,2,1,Suncorp Stadium,City,Australia,Oceania,A-League,League,1.0,06,00,11,Saturday,01,2020,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9963,180096.0,3185.0,Brisbane Roar FC,202.0,Wellington Phoenix,1,0,0,0,Suncorp Stadium,City,Australia,Oceania,A-League,League,1.0,06,00,18,Saturday,01,2020,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9974,189261.0,3185.0,Brisbane Roar FC,1006.0,Adelaide United,2,1,0,0,Suncorp Stadium,City,Australia,Oceania,A-League,League,1.0,08,30,08,Saturday,02,2020,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9982,191439.0,3185.0,Brisbane Roar FC,201.0,Perth Glory,1,1,0,1,Suncorp Stadium,City,Australia,Oceania,A-League,League,1.0,06,00,22,Saturday,02,2020,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9990,194150.0,3185.0,Brisbane Roar FC,199.0,Western Sydney Wanderers FC,3,1,2,0,Suncorp Stadium,City,Australia,Oceania,A-League,League,1.0,09,30,06,Friday,03,2020,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9995,195449.0,3185.0,Brisbane Roar FC,3135.0,Central Coast Mariners,1,0,1,0,Suncorp Stadium,City,Australia,Oceania,A-League,League,1.0,08,30,13,Friday,03,2020,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [144]:
updateLocation(480, 'Newcastle Jets', 'McDonald Jones Stadium', 'Newcastle')

In [149]:
t3 = matches[matches['city'] == 'City']
t3
#matches.loc[matches.competition_name == 'Europe', 'country'] = 'Europe'
#matches[matches['competition_name'] == 'Europe']
#matches[matches.duplicated(['home_id','away_id','day','month','year'])]
list_of_teams = matches.home_id.unique()

In [151]:
list_of_ids = []
for x in list_of_teams:
    ids = matches[matches['home_id'] == x].home_name.unique()
    if(len(ids) > 1):
        list_of_ids.append([x,ids])
list_of_ids

/home/ricardo/.local/lib/python3.7/site-packages/pandas/core/computation/expressions.py:68: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return op(a, b)
/home/ricardo/.local/lib/python3.7/site-packages/pandas/core/computation/expressions.py:68: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return op(a, b)
/home/ricardo/.local/lib/python3.7/site-packages/pandas/core/computation/expressions.py:68: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return op(a, b)
/home/ricardo/.local/lib/python3.7/site-packages/pandas/core/computation/expressions.py:68: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return op(a, b)
/home/ricardo/.local/lib/python3.7/site-packages/pandas/

/home/ricardo/.local/lib/python3.7/site-packages/pandas/core/computation/expressions.py:68: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return op(a, b)
/home/ricardo/.local/lib/python3.7/site-packages/pandas/core/computation/expressions.py:68: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return op(a, b)
/home/ricardo/.local/lib/python3.7/site-packages/pandas/core/computation/expressions.py:68: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return op(a, b)
/home/ricardo/.local/lib/python3.7/site-packages/pandas/core/computation/expressions.py:68: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return op(a, b)
/home/ricardo/.local/lib/python3.7/site-packages/pandas/

[]

In [15]:
matches[matches['got_stats'] == 'T']

,match_id,home_id,home_name,away_id,away_name,home_ft_goals,away_ft_goals,home_ht_goals,away_ht_goals,stadium_name,city,country,continent,competition_name,type,tier,hour,minute,day,dayofweek,month,year,got_stats,home_yc,away_yc,home_rc,away_rc,home_possession,away_possession,home_shots,away_shots,home_shots_on_target,away_shots_on_target,home_fouls,away_fouls,home_attacks,away_attacks
0,152587.0,7.0,Liverpool,492.0,Norwich City,4,1,0.0,0.0,Anfield,Liverpool,England,Europe,Premier League,League,1.0,19.0,0.0,9.0,Friday,8.0,2019.0,T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,152653.0,14.0,West Ham United,12.0,Manchester City,0,5,0.0,1.0,London Stadium,London,England,Europe,Premier League,League,1.0,11.0,30.0,10.0,Saturday,8.0,2019.0,T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,152730.0,1.0,AFC Bournemouth,281.0,Sheffield United,1,1,0.0,0.0,Vitality Stadium,Bournemouth,England,Europe,Premier League,League,1.0,14.0,0.0,10.0,Saturday,8.0,2019.0,T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,152731.0,8.0,Burnley,4.0,Southampton,3,0,0.0,0.0,Turf Moor,Burnley,England,Europe,Premier League,League,1.0,14.0,0.0,10.0,Saturday,8.0,2019.0,T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,152732.0,3.0,Crystal Palace,20.0,Everton,0,0,0.0,0.0,Selhurst Park,London,England,Europe,Premier League,League,1.0,14.0,0.0,10.0,Saturday,8.0,2019.0,T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9912,170822.0,201.0,Perth Glory,3135.0,Central Coast Mariners,1,2,0.0,0.0,HBF Park,Perth,Australia,Oceania,A-League,League,1.0,7.0,0.0,3.0,Sunday,11.0,2019.0,T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9913,171337.0,200.0,Melbourne City FC,3135.0,Central Coast Mariners,3,1,1.0,0.0,AAMI Park,Melbourne,Australia,Oceania,A-League,League,1.0,8.0,30.0,8.0,Friday,11.0,2019.0,T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9914,171521.0,480.0,Newcastle Jets,201.0,Perth Glory,1,1,1.0,0.0,McDonald Jones Stadium,Newcastle,Australia,Oceania,A-League,League,1.0,6.0,0.0,9.0,Saturday,11.0,2019.0,T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9915,171524.0,4545.0,Western United,199.0,Western Sydney Wanderers FC,2,1,1.0,1.0,GMHBA Stadium,Melbourne,Australia,Oceania,A-League,League,1.0,8.0,30.0,9.0,Saturday,11.0,2019.0,T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
